In [125]:
import game_data, fbref_functions

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

In [126]:
home_team = "liverpool"
away_team = "everton"
season = 2024
competition = "england"

In [127]:
import scores_and_fixtures

In [128]:
dir(scores_and_fixtures)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__warningregistry__',
 'season_results_and_fixtures']

In [129]:
scores_and_fixtures.season_results_and_fixtures("eng", 2020)

,gameweek,dayofweek,date,start_time,home_team,home_xg,home_score,away_score,away_xg,away_team,attendance,venue,referee,match_report,notes,match_url
0,1.0,Sat,2020-09-12,12:30,Fulham,0.1,0,3,1.9,Arsenal,0,Craven Cottage,Chris Kavanagh,Match Report,NaN,/en/matches/bf52349b/Fulham-Arsenal-September-...
1,1.0,Sat,2020-09-12,15:00,Crystal Palace,1.1,1,0,0.9,Southampton,0,Selhurst Park,Jonathan Moss,Match Report,NaN,/en/matches/db261cb0/Crystal-Palace-Southampto...
2,1.0,Sat,2020-09-12,17:30,Liverpool,2.7,4,3,0.3,Leeds United,0,Anfield,Michael Oliver,Match Report,NaN,/en/matches/21b58926/Liverpool-Leeds-United-Se...
3,1.0,Sat,2020-09-12,20:00,West Ham,1.0,0,2,1.6,Newcastle Utd,0,London Stadium,Stuart Attwell,Match Report,NaN,/en/matches/78495ced/West-Ham-United-Newcastle...
4,1.0,Sun,2020-09-13,14:00,West Brom,0.4,0,3,2.8,Leicester City,0,The Hawthorns,Anthony Taylor,Match Report,NaN,/en/matches/7dd01ca9/West-Bromwich-Albion-Leic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2021-05-23,16:00,West Ham,1.3,3,0,1.5,Southampton,10000,London Stadium,Martin Atkinson,Match Report,NaN,/en/matches/50ec33b2/West-Ham-United-Southampt...
376,38.0,Sun,2021-05-23,16:00,Leeds United,2.3,3,1,1.0,West Brom,8000,Elland Road,David Coote,Match Report,NaN,/en/matches/9ade2b5d/Leeds-United-West-Bromwic...
377,38.0,Sun,2021-05-23,16:00,Arsenal,0.8,2,0,0.2,Brighton,10000,Emirates Stadium,Jonathan Moss,Match Report,NaN,/en/matches/9f6c52f8/Arsenal-Brighton-and-Hove...
378,38.0,Sun,2021-05-23,16:00,Liverpool,1.6,2,0,0.6,Crystal Palace,9901,Anfield,Craig Pawson,Match Report,NaN,/en/matches/c2fc07f0/Liverpool-Crystal-Palace-...


In [131]:
home_df = game_data.home_player_stats(competition, i, home_team, away_team)
away_df = game_data.away_player_stats(competition, i, home_team, away_team)

KeyError: 'attendance'

In [ ]:
home_df['team'] = home_team
away_df['team'] = away_team

In [ ]:
full_df = pd.concat([home_df, away_df])

In [ ]:
plt.figure(figsize = (15,8), dpi = 200, facecolor = '#f4f2e5')

ax = plt.axes()
ax.grid(True, c = "grey", ls = ":")
ax.set_facecolor("#f4f2e5")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.ylim([0, 110])
plt.xlim([-0.05, 150])

plt.title(f"pass quality - {season} - {home_team} - {away_team}", fontsize = 20, y = 1.01) 

ax.set_ylabel(f"passes_percentage", fontsize = 15)
ax.set_xlabel(f"passes_completed", fontsize = 15)

liverpool = full_df.loc[full_df['team'] == "liverpool"]
everton = full_df.loc[full_df['team'] == "everton"]

ax.scatter(liverpool['passes_completed'], liverpool['passes_pct'], s = liverpool['minutes'] * 2, c = "red", ec = "black", alpha = 0.5, label = "Premier League", zorder=3)
ax.scatter(everton['passes_completed'], everton['passes_pct'], s = everton['minutes'] * 2, c = "blue", ec = "black", alpha = 0.5, label = "Premier League", zorder=3)

for i, row in full_df.iterrows():
    ax.text(row['passes_completed'] + 1, row['passes_pct'], row['player'])

In [132]:
def home_player_stats(comp, year, home_team, away_team):
    
    import scores_and_fixtures
    import check_name
    from get_position import get_basic_position, get_detailed_position
    from datetime import datetime
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    from collections import defaultdict

    results_df = scores_and_fixtures.season_results_and_fixtures(comp, year)
    home_team = check_name.get_team_name(home_team)
    away_team = check_name.get_team_name(away_team)
    
    for i, row in results_df.iterrows():
        if row['home_team'] == home_team and row['away_team'] == away_team:
            url = f"https://fbref.com/{row['match_url']}"
            resp = requests.get(url) 
            resp.encoding = 'utf-8'
            soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
            
            tables = soup.find_all('table', {'class':'stats_table sortable'})
            tbodies = soup.find_all('tbody')
            
            home_team_id = tables[0].get('id').split('_')[1]
            away_team_id = tables[-1].get('id').split('_')[1]
            
            for table in tables:
                if table.get('id').split('_')[1] == home_team_id:
                    player_ids, player_names = [], []
                    
                    th = table.find_all('th', {'data-stat':'player'})
                    
                    for h in th:
                        if player_ids.append(h.get('data-append-csv')) not in player_ids:
                            player_ids.append(h.get('data-append-csv'))
                        if player_names.append(h.get('csk')) not in player_names:
                            player_names.append(h.get('csk'))
                            
                    df = pd.DataFrame(list(zip(player_ids, player_names)), columns = ['player_id', 'player'])
                    df = df.drop(df.index[0]).reset_index(drop = True)
                    df = df[:-1].reset_index(drop = True)
            
            for table in tables:
                if table.get('id').split('_')[1] == home_team_id:
                    headers, full_player_values = [], []
                    rows = table.find_all('tr')
                    
                    row_for_headers = rows[-1].find_all('td')
                    
                    for value in row_for_headers:
                        headers.append(value.get('data-stat'))
                    
                    for row in rows:
                        player_values = []
                        data = row.find_all('td')
                        for value in data:
                            player_values.append(value.get_text())
                        full_player_values.append(player_values)
                        
                    data_df = pd.DataFrame(full_player_values, columns = headers)
                    data_df = data_df.dropna()
                    data_df = data_df[:-1].reset_index(drop = True)
                    
                    df = pd.concat([df, data_df], axis = 1)
                    
            df = df.loc[:, ~df.columns.duplicated()]
            
            df = df.apply(pd.to_numeric, errors = 'ignore') 
            
            df = df.fillna(0)
            
            return df

In [133]:
home_player_stats("eng", 2020, "arsenal", "man city")

,player_id,player,shirtnumber,nationality,position,age,minutes,goals,assists,pens_made,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,d5dd5f1f,Aubameyang Pierre-Emerick,14,ga GAB,"FW,LW",31-248,90,0,0,0,...,0,1,1,0,0,0,1,1,2,33.3
1,bc7dc64d,Saka Bukayo,7,eng ENG,"LW,RW",19-169,90,0,0,0,...,1,3,0,0,0,0,2,0,0,0.0
2,57e3f0c7,Pépé Nicolas,19,ci CIV,RW,25-268,72,0,0,0,...,0,0,0,0,0,0,3,0,1,0.0
3,17695062,Smith Rowe Emile,32,eng ENG,AM,20-208,18,0,0,0,...,1,0,0,0,0,0,0,0,0,0.0
4,79300479,Ødegaard Martin,11,no NOR,AM,22-066,72,0,0,0,...,0,0,0,0,0,0,4,0,2,0.0
5,9dbb75ca,Lacazette Alexandre,9,fr FRA,FW,29-269,18,0,0,0,...,0,2,0,0,0,0,0,1,0,100.0
6,e61b8aee,Xhaka Granit,34,ch SUI,DM,28-147,90,0,0,0,...,3,0,0,0,0,0,5,1,0,100.0
7,47a34c4f,Elneny Mohamed,25,eg EGY,DM,28-225,85,0,0,0,...,1,0,0,0,0,0,5,0,0,0.0
8,c0617e2b,Ceballos Dani,8,es ESP,DM,24-198,5,0,0,0,...,1,1,0,0,0,0,0,0,0,0.0
9,fce2302c,Tierney Kieran,3,sct SCO,LB,23-261,90,0,0,0,...,0,1,0,0,0,0,5,0,0,0.0
